In [124]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [125]:
from google.colab import drive
drive.mount('/content/mnt')

Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


In [14]:
result_dir = '/content/mnt/MyDrive/results/CoffeeIndex'


In [126]:
starbucks = pd.read_csv(f'{result_dir}/스타벅스.csv')
coffeebean = pd.read_csv(f'{result_dir}/커피빈.csv' )
ediya = pd.read_csv(f'{result_dir}/이디야.csv' )
paik = pd.read_csv(f'{result_dir}/빽다방.csv' )


## ID 필드생성(구별, 시별, 도로명주소별)

In [127]:
starbucks.head(2)

,상호명,지점명,시도명,시군구명,도로명주소,상권업종중분류코드
0,스타벅스,천안이마트점,충청남도,천안시 서북구,충청남도 천안시 서북구 충무로 187,Q12
1,스타벅스천안ICDT점,천안ICDT점,충청남도,천안시 동남구,충청남도 천안시 동남구 신부15길 3,Q12


In [128]:
tmp_gu_dict = {
    '수원':['장안구', '권선구', '팔달구', '영통구'], 
    '성남':['수정구', '중원구', '분당구'], 
    '안양':['만안구', '동안구'], 
    '안산':['상록구', '단원구'], 
    '고양':['덕양구', '일산동구', '일산서구'], 
    '용인':['처인구', '기흥구', '수지구'], 
    '청주':['상당구', '서원구', '흥덕구', '청원구'], 
    '천안':['동남구', '서북구'], 
    '전주':['완산구', '덕진구'], 
    '포항':['남구', '북구'], 
    '창원':['의창구', '성산구', '진해구', '마산합포구', '마산회원구']
}

In [244]:
def siname(pop):
  metro_list = ['서울특별시','부산광역시','대구광역시','인천광역시','대전광역시','광주광역시','울산광역시']
  si_name = [None] * len(pop)

  for i in pop.index:
      if pop.시도명[i] in metro_list:
          if len(pop.시군구명[i]) == 2:
              si_name[i] = pop.시도명[i][:2] + ' ' + pop.시군구명[i]
          else:
              si_name[i] = pop.시도명[i][:2] + ' ' + pop.시군구명[i][:-1]     # 긴 구 이름에서 '구' 제외
      else:
          si_len = len(pop.시군구명[i].split())
          if si_len == 1:
            if pop.시군구명[i][:-1] == '고성':
                if pop.시도명[i] == '강원도':
                    si_name[i] = '고성(강원)'
                else:
                    si_name[i] = '고성(경남)'
            elif pop.시군구명[i][:2] == '세종':
              si_name[i] = '세종'
            elif pop.시군구명[i] in ['수원시', '안산시', '용인시']:
              si_name[i] = pop.시군구명[i][:2] + ' ' + pop.도로명주소[i].split()[2]
            elif pop.시군구명[i] in '창원시':
              si_name[i] = pop.시군구명[i][:2] + ' ' + '합포'
            else:
                si_name[i] = pop.시군구명[i][:-1]

          else:
            _, admingu = pop.시군구명[i].split()
            for key, values in tmp_gu_dict.items():
                if admingu in values:
                  if len(admingu) ==2:
                    si_name[i] = key + ' ' +  admingu
                  elif admingu in ['마산합포구', '마산회원구']:
                    si_name[i] = key + ' ' + admingu[2:-1]
                  else:
                    si_name[i] = key + ' ' + admingu[:-1]
  return si_name

In [249]:
sb_si_name = siname(starbucks)
cb_si_name = siname(coffeebean)
ed_si_name = siname(ediya)
pk_si_name = siname(paik)

In [250]:
starbucks['id'] = sb_si_name
coffeebean['id'] = cb_si_name
ediya['id'] = ed_si_name
paik['id'] = pk_si_name

In [251]:
ediya[ediya.id=='창원 합포']

,상호명,지점명,시도명,시군구명,도로명주소,상권업종중분류코드,id
1127,이디야커피,마산경남데파트점,경상남도,창원시 마산합포구,경상남도 창원시 마산합포구 315대로 255,Q12,창원 합포
1130,이디야커피,마산역광장점,경상남도,창원시,경상남도 창원시 마산회원구 마산역광장로 12-1,Q12,창원 합포
1153,이디야커피,마산불종거리점,경상남도,창원시 마산합포구,경상남도 창원시 마산합포구 불종거리로 27,Q12,창원 합포
1156,이디야커피,경남대점,경상남도,창원시 마산합포구,경상남도 창원시 마산합포구 월영동서로 20,Q12,창원 합포
1181,이디야커피,마산진동점,경상남도,창원시 마산합포구,경상남도 창원시 마산합포구 진동면 삼진의거대로 619,Q12,창원 합포
1194,이디야커피,NaN,경상남도,창원시 마산합포구,경상남도 창원시 마산합포구 315대로 156,Q12,창원 합포
1196,이디야커피,마산어시장점,경상남도,창원시 마산합포구,경상남도 창원시 마산합포구 해안대로 331,Q12,창원 합포


In [252]:
ediya.시군구명[0].split()

['천안시', '동남구']

In [253]:
sb = starbucks.groupby(['id'])[['도로명주소']].count()
sb.columns = ['sb매장수']

cb = coffeebean.groupby(['id'])[['도로명주소']].count()
cb.columns = ['cb매장수']

ed = ediya.groupby(['id'])[['도로명주소']].count()
ed.columns = ['ed매장수']

pk = paik.groupby(['id'])[['도로명주소']].count()
pk.columns = ['pk매장수']

In [254]:
sb.reset_index(inplace=True)
cb.reset_index(inplace=True)
ed.reset_index(inplace=True)
pk.reset_index(inplace=True)

In [255]:
#all_coffee = pd.concat([sb,cb, ed, pk]).fillna(0)

sb_cb = pd.merge(sb,cb, how="outer", on= 'id')

In [256]:
ed_pk = pd.merge(ed,pk, how="left", on= 'id')

In [257]:
sb_cb.head()

,id,sb매장수,cb매장수
0,가평,2.0,NaN
1,강릉,5.0,NaN
2,거제,6.0,NaN
3,경산,8.0,NaN
4,경주,8.0,2.0


In [258]:
ed_pk.head()

,id,ed매장수,pk매장수
0,가평,9,NaN
1,강릉,15,2.0
2,강진,1,NaN
3,거제,9,NaN
4,거창,1,1.0


In [259]:
all_coffee = pd.merge(ed_pk,sb_cb, how="left", on= 'id').fillna(0)

In [260]:
all_coffee

,id,ed매장수,pk매장수,sb매장수,cb매장수
0,가평,9,0.0,2.0,0.0
1,강릉,15,2.0,5.0,0.0
2,강진,1,0.0,0.0,0.0
3,거제,9,0.0,6.0,0.0
4,거창,1,1.0,0.0,0.0
...,...,...,...,...,...
235,홍천,3,1.0,2.0,0.0
236,화성,28,10.0,16.0,5.0
237,화순,1,1.0,0.0,0.0
238,화천,1,1.0,0.0,0.0


In [261]:
c_sum = []
for idx, cf in all_coffee.iterrows():
  if (cf.sb매장수 + cf.cb매장수) ==0 or (cf.ed매장수 + cf.pk매장수) ==0 :
    c = 0
  else:
    c = (cf.sb매장수 + cf.cb매장수) / (cf.ed매장수 + cf.pk매장수)
  c_sum.append(c)


In [262]:
all_coffee['커피지수'] = c_sum

In [263]:
all_coffee

,id,ed매장수,pk매장수,sb매장수,cb매장수,커피지수
0,가평,9,0.0,2.0,0.0,0.222222
1,강릉,15,2.0,5.0,0.0,0.294118
2,강진,1,0.0,0.0,0.0,0.000000
3,거제,9,0.0,6.0,0.0,0.666667
4,거창,1,1.0,0.0,0.0,0.000000
...,...,...,...,...,...,...
235,홍천,3,1.0,2.0,0.0,0.500000
236,화성,28,10.0,16.0,5.0,0.552632
237,화순,1,1.0,0.0,0.0,0.000000
238,화천,1,1.0,0.0,0.0,0.000000


In [264]:
all_coffee.to_csv(f'{result_dir}/전국커피지수.csv', index=False)